In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [2]:
dataset_dir = r"D:\DermaEvolve_MAJOR\4_SMOTE_OS\Synthetic_Images_unprocessed"
image_size = (64, 64)
batch_size = 32
num_classes = len(os.listdir(dataset_dir))

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 101127 images belonging to 13 classes.


In [5]:
val_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 25272 images belonging to 13 classes.


In [6]:
def predict_image(model, image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0 

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    
    class_labels = list(val_generator.class_indices.keys())
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label, predicted_class_index

In [7]:
model_paths = {
    "CNN_Customized": r"D:\DermaEvolve_MAJOR\MODELS\CNN_Customized\dermaevolve_custom_cnn_model.keras",
    "DenseNet_169": r"D:\DermaEvolve_MAJOR\MODELS\DenseNet_169\densenet_169_model.keras",
    "MobileNet": r"D:\DermaEvolve_MAJOR\MODELS\MobileNet\mobilenet_model.keras",
    "NasNet": r"D:\DermaEvolve_MAJOR\MODELS\NasNet\nasnet_model.keras",
    "ResNet_50" : r"D:\DermaEvolve_MAJOR\MODELS\ResNet_50\resnet_50_model.keras"
}

In [12]:
def image_iterate(image_path, actual):

    for model_name, model_path in model_paths.items():
        model = tf.keras.models.load_model(model_path)
        print("-------------------------------------------------------------------------------------------------")
        print(model_name)
        start = time.time()
        predicted_class_label, predicted_class_index = predict_image(model, image_path)
        end = time.time()
        elapsed_time = end - start
        if predicted_class_label == actual:
            correctness = "✅"
        else:
            correctness = "❌"
        print(f"Time Consumed = {elapsed_time:.2f} seconds")
        print(f"Predicted Class: {predicted_class_label}, Class Index: {predicted_class_index} -----> ", correctness)

In [13]:
image_path = r"D:\DermaEvolve_MAJOR\4_SMOTE_OS\Synthetic_Images_unprocessed\Dermatofibroma\smote_3786.jpg"
actual = "Dermatofibroma"
image_iterate(image_path, actual)

-------------------------------------------------------------------------------------------------
CNN_Customized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
Time Consumed = 0.41 seconds
Predicted Class: Dermatofibroma, Class Index: 3 ----->  ✅
-------------------------------------------------------------------------------------------------
DenseNet_169
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.13 seconds
Predicted Class: Dermatofibroma, Class Index: 3 ----->  ✅
-------------------------------------------------------------------------------------------------
MobileNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Time Consumed = 1.08 seconds
Predicted Class: Dermatofibroma, Class Index: 3 ----->  ✅
-------------------------------------------------------------------------------------------------
NasNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.47 seconds
Predicted Class: Dermatofibroma, Class Index: 3 ----->  ✅
--------------------------------------------------------------------

In [14]:
image_path = r"D:\DermaEvolve_MAJOR\Test_Images\Melanocytic_Nevus.jpg"
actual = "Melanocytic_Nevus"
image_iterate(image_path, actual)

-------------------------------------------------------------------------------------------------
CNN_Customized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
Time Consumed = 0.38 seconds
Predicted Class: Melanocytic_Nevus, Class Index: 6 ----->  ✅
-------------------------------------------------------------------------------------------------
DenseNet_169
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.42 seconds
Predicted Class: Melanoma, Class Index: 7 ----->  ❌
-------------------------------------------------------------------------------------------------
MobileNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Time Consumed = 1.14 seconds
Predicted Class: Melanocytic_Nevus, Class Index: 6 ----->  ✅
-------------------------------------------------------------------------------------------------
NasNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.51 seconds
Predicted Class: Melanocytic_Nevus, Class Index: 6 ----->  ✅
-----------------------------------------------------------------

In [15]:
image_path = r"D:\DermaEvolve_MAJOR\Test_Images\Melanoma.jpg"
actual = "Melanoma"
image_iterate(image_path, actual)

-------------------------------------------------------------------------------------------------
CNN_Customized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step
Time Consumed = 0.44 seconds
Predicted Class: Melanoma, Class Index: 7 ----->  ✅
-------------------------------------------------------------------------------------------------
DenseNet_169
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.47 seconds
Predicted Class: Melanoma, Class Index: 7 ----->  ✅
-------------------------------------------------------------------------------------------------
MobileNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Time Consumed = 1.50 seconds
Predicted Class: Pigmented_Benign_Keratosis, Class Index: 9 ----->  ❌
-------------------------------------------------------------------------------------------------
NasNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.24 seconds
Predicted Class: Seborrheic_Keratosis, Class Index: 10 ----->  ❌
-------------------------------------------------------------

In [16]:
image_path = r"D:\DermaEvolve_MAJOR\4_SMOTE_OS\Synthetic_Images_unprocessed\Dermatofibroma\smote_3786.jpg"
actual = "Dermatofibroma"
image_iterate(image_path, actual)

-------------------------------------------------------------------------------------------------
CNN_Customized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step
Time Consumed = 0.42 seconds
Predicted Class: Dermatofibroma, Class Index: 3 ----->  ✅
-------------------------------------------------------------------------------------------------
DenseNet_169
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.12 seconds
Predicted Class: Dermatofibroma, Class Index: 3 ----->  ✅
-------------------------------------------------------------------------------------------------
MobileNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Time Consumed = 1.50 seconds
Predicted Class: Dermatofibroma, Class Index: 3 ----->  ✅
-------------------------------------------------------------------------------------------------
NasNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
Time Consumed = 9.75 seconds
Predicted Class: Dermatofibroma, Class Index: 3 ----->  ✅
------------------------------------------------------------------

In [17]:
image_path = r"D:\DermaEvolve_MAJOR\Test_Images\Blue_Naevus_4.jpg"
actual = "Blue_Naevus"
image_iterate(image_path, actual)

-------------------------------------------------------------------------------------------------
CNN_Customized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
Time Consumed = 0.36 seconds
Predicted Class: Blue_Naevus, Class Index: 2 ----->  ✅
-------------------------------------------------------------------------------------------------
DenseNet_169
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 8.57 seconds
Predicted Class: Blue_Naevus, Class Index: 2 ----->  ✅
-------------------------------------------------------------------------------------------------
MobileNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Time Consumed = 1.09 seconds
Predicted Class: Blue_Naevus, Class Index: 2 ----->  ✅
-------------------------------------------------------------------------------------------------
NasNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.03 seconds
Predicted Class: Blue_Naevus, Class Index: 2 ----->  ✅
--------------------------------------------------------------------------------

In [18]:
image_path = r"D:\DermaEvolve_MAJOR\Test_Images\Elastosis_Perforans_Serpiginosa_4.jpg"
actual = "Elastosis_Perforans_Serpiginosa"
image_iterate(image_path, actual)

-------------------------------------------------------------------------------------------------
CNN_Customized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
Time Consumed = 0.38 seconds
Predicted Class: Elastosis_Perforans_Serpiginosa, Class Index: 4 ----->  ✅
-------------------------------------------------------------------------------------------------
DenseNet_169
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.30 seconds
Predicted Class: Elastosis_Perforans_Serpiginosa, Class Index: 4 ----->  ✅
-------------------------------------------------------------------------------------------------
MobileNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Time Consumed = 1.06 seconds
Predicted Class: Elastosis_Perforans_Serpiginosa, Class Index: 4 ----->  ✅
-------------------------------------------------------------------------------------------------
NasNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.31 seconds
Predicted Class: Elastosis_Perforans_Serpiginosa, Class Index: 4 ----->  ✅


In [19]:
image_path = r"D:\DermaEvolve_MAJOR\Test_Images\Elastosis_Perforans_Serpiginosa_5.jpg"
actual = "Elastosis_Perforans_Serpiginosa"
image_iterate(image_path, actual)

-------------------------------------------------------------------------------------------------
CNN_Customized
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
Time Consumed = 0.39 seconds
Predicted Class: Elastosis_Perforans_Serpiginosa, Class Index: 4 ----->  ✅
-------------------------------------------------------------------------------------------------
DenseNet_169
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 9.50 seconds
Predicted Class: Elastosis_Perforans_Serpiginosa, Class Index: 4 ----->  ✅
-------------------------------------------------------------------------------------------------
MobileNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Time Consumed = 1.24 seconds
Predicted Class: Elastosis_Perforans_Serpiginosa, Class Index: 4 ----->  ✅
-------------------------------------------------------------------------------------------------
NasNet
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Time Consumed = 8.71 seconds
Predicted Class: Elastosis_Perforans_Serpiginosa, Class Index: 4 ----->  ✅
